# model TPH

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
#import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
# Import Halving Grid Search
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
import xgboost as xgb
from scipy.stats import kurtosis,skew
from numpy import mean,sqrt,square
from scipy.fftpack import fft, fftfreq
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# Función que filtra tags por alta correlación
def corrkill(dataframe, features, corr_cut=0.8):

    df = dataframe[features]
    dfcorr = pd.DataFrame(np.triu(df.corr()), columns=df.columns, index=df.columns)
    dfcorr = dfcorr.stack().reset_index()
    dfcorr.columns = ['Feat1','Feat2','Val']
    dfcorr = dfcorr[~dfcorr['Val'].isin([0,1])]
    dfcorr = dfcorr[dfcorr['Val'].abs()>corr_cut]
    dfcorr["Val"]=dfcorr["Val"].abs()
    
    return list(dfcorr['Feat2']), dfcorr.sort_values(by="Val",ascending=False)

In [3]:
variables_interes=['pyrite law', 'chalcopyrite law',
       'chalcocite law', 'covelin law', 'crusher index', 'sag power index',
       'ball work index', 'bornite law','charge cell', 'TPH','water', 'speed','HH TPH', 'granulometry', 'Edad','loss of TPH',
       "power","solid percentage","HH charge cell","LL charge cell","delta HH TPH","delta HH charge cell","delta LL charge cell"]
variables_interes

['pyrite law',
 'chalcopyrite law',
 'chalcocite law',
 'covelin law',
 'crusher index',
 'sag power index',
 'ball work index',
 'bornite law',
 'charge cell',
 'TPH',
 'water',
 'speed',
 'HH TPH',
 'granulometry',
 'Edad',
 'loss of TPH',
 'power',
 'solid percentage',
 'HH charge cell',
 'LL charge cell',
 'delta HH TPH',
 'delta HH charge cell',
 'delta LL charge cell']

In [4]:
# read
tags=pd.read_excel("../../../data/meta data/tags relevantes.xlsx")
tags_cc=tags.tag.to_list()
dic={}
for i,j in zip(tags.tag,tags.description):
    dic[i]=j

In [ ]:
# Se lee csv
liners_age=pd.read_csv('../../../data/consolidated data/liners_age_03January2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
# Se lee csv
cleaned=pd.read_csv('../../../data/consolidated data/cleaned_15December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
cleaned.sort_index(inplace=True)
cleaned.rename(columns=dic,inplace=True)
df_cleaned_age=cleaned.join(liners_age).dropna()
df_cleaned_age["delta HH TPH"]=df_cleaned_age["HH TPH"]-df_cleaned_age["TPH"]
df_cleaned_age["delta HH charge cell"]=df_cleaned_age["HH charge cell"]-df_cleaned_age["charge cell"]
df_cleaned_age["delta LL charge cell"]=df_cleaned_age["charge cell"]-df_cleaned_age["LL charge cell"]
df_cleaned_age["loss of TPH"]=df_cleaned_age["delta HH TPH"].apply(lambda x: 1 if x>100 else 0)
df_cleaned_age=df_cleaned_age[variables_interes]
df_cleaned_age.head()

In [ ]:
df_cleaned_age["TPH"].ewm(span=5,min_periods=0,ignore_na=True).mean()

In [ ]:
df_corr = df_cleaned_age.dropna().corr()
corr_tph=pd.DataFrame(df_corr.abs().TPH.sort_values(ascending=False))
top_corr=corr_tph[corr_tph.TPH>=0]
top_corr

# Feature Engineering

In [5]:
def feature_engineering(df):
      
    targets = ["TPH","Edad","loss of TPH","HH TPH"]
    df_vars = df.copy()
    column_inicial=df_vars.columns

    for column in df_vars:
    
        if (column not in targets):

            ## 10 min
            #df_vars[f"skew_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:skew(x))
            #df_vars[f"kurt_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:kurtosis(x))
            df_vars[f"mean_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_10"]=df_vars[column].rolling(window=10).apply(lambda x: np.nanvar(x))

            # 5 min
            df_vars[f"mean_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_5"]=df_vars[column].rolling(window=5).apply(lambda x: np.nanvar(x))

            # 3 min
            #df_vars[f"skew_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:skew(x))
            #df_vars[f"kurt_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:kurtosis(x))
            df_vars[f"mean_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_3"]=df_vars[column].rolling(window=3).apply(lambda x: np.nanvar(x))
          

    for column in column_inicial:

        if (column not in targets):
            
            # lags  
            df_vars[f"{column}_(t-1)"] = df_vars[column].shift(1)
            df_vars[f'{column}_(t-2)'] = df_vars[column].shift(2)
            df_vars[f'{column}_(t-3)'] = df_vars[column].shift(3)
            df_vars[f'{column}_(t-4)'] = df_vars[column].shift(4)
            df_vars[f'{column}_(t-5)'] = df_vars[column].shift(5)
           
    
    return df_vars
   

In [ ]:
df_features=feature_engineering(df_cleaned_age)

In [ ]:
# Se guarda en csv
df_features.to_csv('../../../data/processed data/df_feature_engineering_'+pd.to_datetime('today').strftime('%d%B%Y')+'.csv', index=True)

In [6]:
# read df full csv
df_features0=pd.read_csv('../../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#["2020":"2021"]
df_features0["TPH"]=df_features0["TPH"].shift(-10)
df_features0["loss of TPH"]=df_features0["loss of TPH"].shift(-10)
df_features0.dropna(inplace=True)
df_features0.sort_index(inplace=True)
df_features0.shape

(859325, 403)

In [13]:
# Verificando si hay duplicados
u=[i for i in df_features0.columns if (not re.match(".*delta HH TPH",i)) and (not re.match(".*water",i)) and (not re.match(".*power",i)) ]
y=[i for i in df_features0.columns if  re.match(".*sag power index",i)] 
u=y+u
df_features=df_features0[u].copy()
df_features.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

# Correlación entre variables

In [8]:
# Se obtienen tags altamente correlacionados y matriz que permite visualizar cuales son los par de tags altamente correlacionados
collinearity, table_cor=corrkill(df_features,df_features.columns)

In [9]:
# Tags altamente correlacionados
table_cor

,Feat1,Feat2,Val
92689,mean_HH charge cell_3,rms_HH charge cell_3,1.000000
98434,mean_LL charge cell_3,rms_LL charge cell_3,1.000000
90774,mean_HH charge cell_5,rms_HH charge cell_5,1.000000
96519,mean_LL charge cell_5,rms_LL charge cell_5,1.000000
58219,mean_charge cell_3,rms_charge cell_3,1.000000
...,...,...,...
66290,max_speed_10,min_power_10,0.820700
70491,min_speed_3,max_power_10,0.817930
66676,min_speed_10,max_power_5,0.815335
68581,min_speed_5,max_power_10,0.809710


In [10]:
df_corr = df_features.dropna().corr()
corr_tph=pd.DataFrame(df_corr.abs().TPH.sort_values(ascending=False))


In [12]:
top_corr=corr_tph[corr_tph.TPH.abs()>=0.25]
top_corr

,TPH
TPH,1.000000
HH TPH,0.639734
loss of TPH,0.583261
min_solid percentage_10,0.344294
min_solid percentage_5,0.344096
min_solid percentage_3,0.343257
solid percentage,0.341291
mean_solid percentage_3,0.340318
rms_solid percentage_3,0.340282
solid percentage_(t-1),0.339998


# Equilibrio de eventos de perdidas de TPH

In [14]:
# Entrenamiento
entrenamiento=pd.concat([df_features["2020-01":"2020-03"],df_features["2020-05"],df_features["2020-07":"2020-12"],df_features["2021-01":"2021-08"],df_features["2021-10"],df_features["2021-12"]])
entrenamiento.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [15]:
# Validación
validacion=pd.concat([df_features["2020-06"],df_features["2021-11"]])
validacion.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [16]:
# Test
test=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
test.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [17]:
tags=df_features.columns.to_list()
tags.remove("loss of TPH")
#tags.remove("TPH")

In [18]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

# ROS:
Xtrain, Ytrain=ros.fit_resample(entrenamiento[tags],entrenamiento["loss of TPH"])

In [19]:
# Ytrain reg
Ytrain=Xtrain["TPH"]
Xtrain=Xtrain.drop(columns=["TPH"])

In [20]:
# Yval reg
Yval=validacion["TPH"]
Xval=validacion.drop(columns=["TPH"])

In [21]:
Ytest0=test[test["loss of TPH"]==0]["TPH"]


In [22]:
Ytest1=test[test["loss of TPH"]==1]["TPH"]


In [23]:
Xtest0=test[test["loss of TPH"]==0].drop(columns=["TPH","loss of TPH"])


In [24]:
Xtest1=test[test["loss of TPH"]==1].drop(columns=["TPH","loss of TPH"])


In [25]:
# Ytest reg
Ytest=test["TPH"]
Xtest=test.drop(columns=["TPH"])

In [26]:
#Unión del conjunto de Entrenamiento y Validación:
Entre_Vali_features=pd.concat([Xval,Xtrain])
Entre_Vali_target=pd.concat([Yval,Ytrain])

#Indices para separar ambos conjuntos:
Entre_indice=np.full(len(validacion),-1)
Vali_indice=np.full(len(entrenamiento),0)
Indices=np.append(Entre_indice,Vali_indice)
x=PredefinedSplit(Indices)

In [27]:
Color_loss=test["loss of TPH"]
Color_loss=Color_loss.astype(str)

# Feature selection

In [28]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        Xtrain.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse'
     ))
    ]
)

In [29]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['sag power index',
                                                   'mean_sag power index_10',
                                                   'max_sag power index_10',
                                                   'min_sag power index_10',
                                                   'rms_sag power index_10',
                                                   'var_sag power index_10',
                                                   'mean_sag power index_5',
                                                   'max_sag power index_5',
                                                   'min_sag power index_5',
                                                   'rms_sag power index_5',
                                                   'var_sag power index_5',
                                                  

**General**

In [30]:
Ypred= pipe.predict(Xtest)

# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.5673087419266196 
 RMSE:  278.1091202528349


In [31]:
df_importance=pd.DataFrame({"tag":Xtrain.columns,"importance":pipe["clf"].feature_importances_}).sort_values(by="importance",ascending=False).reset_index(drop=True)
df_importance.head()

,tag,importance
0,min_solid percentage_5,0.065101
1,min_speed_5,0.054733
2,HH TPH,0.051140
3,min_solid percentage_10,0.037579
4,min_solid percentage_3,0.033583


In [32]:
pd.options.plotting.backend = "plotly"
fig=df_importance["importance"].plot(kind='hist')
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [33]:
# Plot
fig = go.Figure([go.Bar(x=df_importance["tag"], y=df_importance["importance"])])
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [34]:
xgboost_fs=df_importance[:100].tag.to_list()
xgboost_fs

['min_solid percentage_5',
 'min_speed_5',
 'HH TPH',
 'min_solid percentage_10',
 'min_solid percentage_3',
 'min_speed_3',
 'rms_delta LL charge cell_3',
 'chalcocite law_(t-3)',
 'rms_solid percentage_5',
 'rms_sag power index_10',
 'rms_sag power index_3',
 'solid percentage',
 'min_delta LL charge cell_10',
 'max_LL charge cell_10',
 'rms_delta LL charge cell_10',
 'min_HH charge cell_10',
 'covelin law_(t-4)',
 'var_bornite law_3',
 'speed',
 'max_HH charge cell_10',
 'max_ball work index_5',
 'rms_bornite law_10',
 'min_sag power index_5',
 'max_sag power index_10',
 'max_delta HH charge cell_10',
 'chalcopyrite law_(t-3)',
 'rms_delta HH charge cell_3',
 'bornite law_(t-5)',
 'rms_crusher index_10',
 'min_chalcocite law_5',
 'sag power index_(t-4)',
 'mean_delta HH charge cell_3',
 'min_delta HH charge cell_3',
 'var_chalcopyrite law_3',
 'min_sag power index_3',
 'delta LL charge cell',
 'rms_delta HH charge cell_5',
 'crusher index_(t-4)',
 'LL charge cell_(t-1)',
 'min_borni

In [35]:
# Se obtienen tags altamente correlacionados y matriz que permite visualizar cuales son los par de tags altamente correlacionados
collinearity, table_cor=corrkill(Xtrain,xgboost_fs,corr_cut=0.9)

In [ ]:
# Tags altamente correlacionados
table_cor

In [36]:
tag_select = [i for i in xgboost_fs if i not in collinearity]
len(tag_select)

25

In [39]:
tag_select

['min_solid percentage_5',
 'min_speed_5',
 'HH TPH',
 'rms_delta LL charge cell_3',
 'chalcocite law_(t-3)',
 'rms_sag power index_10',
 'min_delta LL charge cell_10',
 'max_LL charge cell_10',
 'covelin law_(t-4)',
 'var_bornite law_3',
 'max_ball work index_5',
 'rms_bornite law_10',
 'chalcopyrite law_(t-3)',
 'rms_crusher index_10',
 'var_chalcopyrite law_3',
 'min_charge cell_10',
 'pyrite law_(t-5)',
 'Edad',
 'var_pyrite law_3',
 'var_solid percentage_3',
 'max_granulometry_10',
 'var_charge cell_10',
 'var_pyrite law_5',
 'var_speed_10',
 'var_solid percentage_5']

In [37]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse'
     ))
    ]
)

In [38]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_solid percentage_5',
                                                   'min_speed_5', 'HH TPH',
                                                   'rms_delta LL charge cell_3',
                                                   'chalcocite law_(t-3)',
                                                   'rms_sag power index_10',
                                                   'min_delta LL charge '
                                                   'cell_10',
                                                   'max_LL charge cell_10',
                                                   'covelin law_(t-4)',
                                                   'var_bornite law_3',
                                                   'max_ball work index_5',
                                                   'rms_bornite law_1

**General**

In [40]:
Ypred= pipe.predict(Xtest)

# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.5141710865991208 
 RMSE:  294.6916630129672


# GridSearch

In [ ]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        #features.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
        #("Selection", SelectPercentile(f_classif, percentile=10)),
        #("PCA",PCA(n_components=150 )), #0.08
     ("reg", xgb.XGBRegressor(seed=1,eval_metric='rmse'))
    ]
)


param_grid_reg = {'reg__learning_rate': list(np.linspace(0.001,1,10)), # Boosting learning rate
                  'reg__gamma':list(np.linspace(0.0001,50,5)), # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                  #'reg__max_depth':[5,10,12,15,20], # Maximum tree depth for base learners.
                  #'reg__min_child_weight':[10,5,1,2,3], # Minimum sum of instance weight(hessian) needed in a child.
                  #'reg__colsample_bytree':[0.5,0.8,1], # Subsample ratio of columns when constructing each tree.
                  'reg__n_estimators': [100,200,300], # Number of gradient boosted trees.   
                  #'reg__reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              #'reg__reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
               #'Selection__percentile': [10,20,30,50,70,80,100]              
                 } 

# Clasificación
gs_reg = HalvingGridSearchCV(pipe, param_grid_reg, scoring='neg_mean_absolute_error',cv=x,refit=True,
                             verbose=10,
                             aggressive_elimination=True
                            )

# Fit
gs_reg.fit(Entre_Vali_features, Entre_Vali_target)


In [ ]:
gs_reg.best_score_

In [ ]:
gs_reg.best_params_

In [ ]:
# General
Ypred= gs_reg.predict(Xtest)
# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

# Train model

In [ ]:
tag_select

In [ ]:
tag_select=['min_water_3',
 'min_solid percentage_10',
 'HH TPH',
 'max_delta LL charge cell_10',
 'power',
 'rms_delta LL charge cell_3',
 'var_delta HH charge cell_10',
 'max_covelin law_10',
 'LL charge cell_(t-2)',
 'min_granulometry_5',
 'max_bornite law_10',
 'min_charge cell_5',
 'chalcocite law_(t-2)',
 'max_sag power index_5',
 'min_speed_3',
 'var_bornite law_3',
 'var_speed_3',
 'min_pyrite law_10',
 'crusher index_(t-5)',
 'var_power_3',
 'var_chalcocite law_3',
 'var_bornite law_5',
 'var_solid percentage_3',
 'var_speed_10',
 'ball work index_(t-1)',
 'var_chalcocite law_5',
 'var_water_3',
 'chalcopyrite law_(t-5)',
 'var_crusher index_10',
 'var_chalcopyrite law_3',
 'var_granulometry_3',
 'var_delta HH charge cell_3']

In [ ]:
gs_reg.best_params_

In [41]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse',gamma= 12.500074999999999, learning_rate=0.112, n_estimators= 100
     
     ))
    ]
)
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_solid percentage_5',
                                                   'min_speed_5', 'HH TPH',
                                                   'rms_delta LL charge cell_3',
                                                   'chalcocite law_(t-3)',
                                                   'rms_sag power index_10',
                                                   'min_delta LL charge '
                                                   'cell_10',
                                                   'max_LL charge cell_10',
                                                   'covelin law_(t-4)',
                                                   'var_bornite law_3',
                                                   'max_ball work index_5',
                                                   'rms_bornite law_1

**General**

In [ ]:
# General
Ypred= pipe.predict(Xtest0)
# Métricas
print('R2: ',r2_score(Ytest0, Ypred),'\n','RMSE: ',mean_squared_error(Ytest0, Ypred, squared=False))

In [ ]:
# General
Ypred= pipe.predict(Xtest1)
# Métricas
print('R2: ',r2_score(Ytest1, Ypred),'\n','RMSE: ',mean_squared_error(Ytest1, Ypred, squared=False))

In [ ]:
Ypred= pipe.predict(Xtest)
# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False),'\n','MAPE :',mean_absolute_percentage_error(Ytest, Ypred)*100)

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=Ypred[:10000], y=Ytest[:10000],
                    mode='markers',
                    name='(Pred,Test)'))

fig.add_trace(go.Scatter(x=[i for i in range(2500,4700)], y=[i for i in range(2500,4700)],
                    mode='markers', name='Identidad'))
                    
fig.update_layout(height=500, width=1200, title_text="Scatter predict/test", xaxis_title="Predict",
    yaxis_title="Test")
fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(y=Ypred[:10000], 
                    mode='lines',name="predict"
                    ))

fig.add_trace(go.Scatter(y=Ytest[:10000],
                    mode='lines',name="Test",#marker=dict(color=list(Color_loss[:1000]))
                    ))

fig.update_layout(height=500, width=1200, title_text="TPH vs TPH predict")
fig.show()

**Histograma del error RMSE y desviación estandar**

In [ ]:
(Ytest-Ypred).abs().std()

In [ ]:
(Ytest-Ypred).abs().mean()

In [ ]:
pd.options.plotting.backend = "plotly"
df_dist_error=pd.DataFrame()
df_dist_error["Error"]=(Ytest-Ypred)#.abs()
fig=df_dist_error["Error"].plot(kind='hist')
fig.update_layout(height=500, width=1200,title="Distribución Error RMSE") 
fig.show()

**2020**

In [ ]:
Ypred= pipe.predict(Xtest["2020"])

# Métricas
print('R2: ',r2_score(Ytest["2020"], Ypred),'\n','RMSE: ',mean_squared_error(Ytest["2020"], Ypred, squared=False))

**2021**

In [ ]:
Ypred= pipe.predict(Xtest["2021"])

# Métricas
print('R2: ',r2_score(Ytest["2021"], Ypred),'\n','RMSE: ',mean_squared_error(Ytest["2021"], Ypred, squared=False))

**2022**

In [ ]:
Ypred= pipe.predict(Xtest["2022-01"])

# Métricas
print('R2: ',r2_score(Ytest["2022-01"], Ypred),'\n','RMSE: ',mean_squared_error(Ytest["2022-01"], Ypred, squared=False))

In [ ]:
Ypred= pipe.predict(Xtest["2022-02"])

# Métricas
print('R2: ',r2_score(Ytest["2022-02"], Ypred),'\n','RMSE: ',mean_squared_error(Ytest["2022-02"], Ypred, squared=False))

# Simulation time series predict

# 2020

In [ ]:
# test 2021
test_time_serie=Xtest["2020"]
test_time_serie["TPH"]=Ytest["2020"]
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data["TPH"],
                    mode='lines',
                    name='TPH'))
fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"],
                    mode='lines',
                    name='TPH predict'))

fig.add_trace(go.Scatter(x=data.index, y=data["HH TPH"],
                    mode='lines',
                    name='HH TPH'))

fig.update_layout(height=500, width=1200, title_text="TPH vs TPH predict"#,xaxis6_rangeslider_visible=True
,#template="plotly_dark"
)                   
fig.show()

In [ ]:
fig.write_html("file.html")

# 2021

In [ ]:
# test 2021
test_time_serie=Xtest["2021"]
test_time_serie["TPH"]=Ytest["2021"]
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data["TPH"],
                    mode='lines',
                    name='TPH'))
fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"],
                    mode='lines',
                    name='TPH predict'))

fig.add_trace(go.Scatter(x=data.index, y=data["HH TPH"],
                    mode='lines',
                    name='HH TPH'))

fig.update_layout(height=500, width=1200, title_text="TPH vs TPH predict"#,xaxis6_rangeslider_visible=True
,#template="plotly_dark"
)                   
fig.show()

# 2022

In [ ]:
# test 2022-01
test_time_serie=Xtest["2022-01"]
test_time_serie["TPH"]=Ytest["2022-01"]
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data["TPH"],
                    mode='lines',
                    name='TPH'))
fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"],
                    mode='lines',
                    name='TPH predict'))

fig.add_trace(go.Scatter(x=data.index, y=data["HH TPH"],
                    mode='lines',
                    name='HH TPH'))

fig.update_layout(height=500, width=1200, title_text="TPH vs TPH predict"#,xaxis6_rangeslider_visible=True
,#template="plotly_dark"
)                   
fig.show()

In [ ]:
# test 2022-02
test_time_serie=Xtest["2022-02"]
test_time_serie["TPH"]=Ytest["2022-02"]
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))
test_time_serie["TPH"]=test_time_serie["TPH"].shift(10)
test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data["TPH"],
                    mode='lines',
                    name='TPH'))
fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"],
                    mode='lines',
                    name='TPH predict'))

fig.add_trace(go.Scatter(x=data.index, y=data["HH TPH"],
                    mode='lines',
                    name='HH TPH'))

fig.update_layout(height=500, width=1200, title_text="TPH vs TPH predict"#,xaxis6_rangeslider_visible=True
,#template="plotly_dark"
)                   
fig.show()